In [2]:
# Import libraries
import pandas as pd
import glob
import numpy as np

# Set path for glob to search from and create a list of all paths to the customer Excel files in sub-folders
path = '/Users/emirkabasoglu/Library/CloudStorage/Box-Box/Customer_Data/Accounts_Work_Folder_Latest'
files = glob.glob(path + '/*/*.xlsx', recursive=True)

# Create list of dataframes through a list comprehension reading each Excel file with a path name in files
dfs = [pd.read_excel(f) for f in files]

# Concatenate all dataframes in dfs into a single dataframe
df_main = pd.concat(dfs).reset_index(drop=True)

/Users/emirkabasoglu/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Cell U2 is marked as a date but the serial value 9422444611 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():
/Users/emirkabasoglu/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Cell U3 is marked as a date but the serial value 9422444611 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():
/Users/emirkabasoglu/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Cell U4 is marked as a date but the serial value 9422444611 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


In [3]:
# CHECK FOR DUPLICATE FILES
df_files = pd.DataFrame(files, columns={'path'})
df_files['files'] = df_files['path'].str.lstrip(to_strip=f'{path}')
df_files['folders'] = df_files['files'].str.split(pat='/')
paths = []
for i in df_files['folders']:
    paths.append(i[0])
dup = {x for x in paths if paths.count(x) > 1}
len(dup)

In [4]:
#df_paths = pd.DataFrame (paths, columns = ['econ'])
#df_paths.to_excel('check.xlsx')

In [5]:
#df_main.to_excel('df_main-dump-23-jun-26.xlsx', index=False)
df_main = df_main.dropna(subset=['currency'])
#df_main.currency.unique()

In [6]:
# Define function to convert values to DKK using defined exchange rates. Run function to add column for item prices converted to DKK.
def currency_conversion(price,currency):
    
    # FY23 FX Rates
    multiplier = {"EUR": 7.45, "USD": 6.86, "GBP": 8.68, "DKK": 1, "AUD": 4.54, "NOK": 0.64, "SEK": 0.63, "ISK": 0.05}
    
    return price * multiplier[currency]

df_main["total_item_price_DKK"] = df_main.apply(lambda x: currency_conversion(x["total_item_price"], x["currency"]), axis=1)

df_main["total_item_price_EUR"] = df_main.apply(lambda x: x["total_item_price_DKK"] / 7.45, axis=1)

In [7]:
# Set up dataframe for firmographic data and add these to the main dataframe.
df_firmographics = pd.read_excel('/Users/emirkabasoglu/Library/CloudStorage/Box-Box/Chargebee/Firmographics/firmographics-latest.xlsx', sheet_name='latest')
df_main = df_main.merge(df_firmographics.iloc[:,1:], on='econ_id', how='left')

# Add size segments
def segment(number_of_employees):
    if number_of_employees <= 249:
        return 'SMB'
    elif number_of_employees in range(250,5000):
        return 'MM'
    else:
        return 'ENT'

df_main['fte_size_segment'] = df_main.apply(lambda x: segment(x['number_of_employees']), axis=1)

In [8]:
# Clean old product names
pattern = '|'.join(['nrr_', 'wp_', 'hub_'])
df_main['product_name']= df_main['product_name'].astype(str)
df_main['product_name'] = df_main['product_name'].str.lower().str.replace(pattern, '').str.strip()

# Set up dataframe with streamlined product names
df_product_list = pd.read_excel('/Users/emirkabasoglu/Library/CloudStorage/Box-Box/Chargebee/Products/product-list-latest.xlsx')

# Merge dataframes to add CB product names
df_main = df_main.merge(df_product_list, how='left', left_on='product_name', right_on='old_name')

df_main['product']= df_main['product'].astype(str)

# Set up and execute function for assigning product groups
def product_group(product_name):
    if product_name.lower().startswith('content'):
        return 'content'
    else:
        return 'software'

df_main['product_group'] = df_main.apply(lambda x: product_group(x['product']), axis=1)

# Set up and execute function for assigning product types
def product_type(product_name):
    infinite_products = ['infinite_play', 'infinite_plus', 'infinite_ee', 'infinite_free', 'infinite_service_api_access_support', 'infinite_service_sso_access_support']
    if product_name in infinite_products:
        return 'infinite'
    elif product_name.startswith('content'):
        return ''
    else:
        return 'people'
df_main['product_type'] = df_main.apply(lambda x: product_type(x['product']), axis=1)

/var/folders/84/np97p6s9023f4v7_n4n1nqk40000gp/T/ipykernel_39228/1174387087.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_main['product_name'] = df_main['product_name'].str.lower().str.replace(pattern, '').str.strip()


In [9]:
# Add lead source information
df_hs = pd.read_excel('/Users/emirkabasoglu/Library/CloudStorage/OneDrive-eloomiAS/Shared_Scripts/GRR_Report/hubspot-crm-exports-customer-ops-won-2023-02-03.xlsx')

/Users/emirkabasoglu/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
def inbound_source(source):
    google = ['PPC - Google AdWords']
    bing = ['PPC - Bing']
    # capterra = ['PPC - Capterra', 'Review DB - Capterra']
    facebook = ['PPC - Facebook']
    linkedin = []
    review = ['PPC - Capterra', 'PPC - Software Advice', 'PPC - Getapp', 'Review DB - eLearning Industry', 'Review DB - People Managing People', 'Review DB - Select Software', 'Review DB - Softwaresuggest']
    other = ['Referral - Other']
    if source in google:
        return 'google'
    elif source in bing:
        return 'bing'
    # elif source in capterra:
    #     return 'capterra'
    elif source in facebook:
        return 'facebook'
    elif source in linkedin:
        return 'linkedin'
    elif source in review:
        return 'review'
    elif source in other:
        return 'other'
    else:
        return np.nan

df_hs['source'] = df_hs.apply(lambda x: inbound_source(x['Inbound Lead Source']), axis=1)

In [11]:
# Merge sorted main df with HS info for lead source
df_main = df_main.merge(df_hs[['Record ID', 'Inbound Lead Source', 'source',]], how='left', left_on='order_id', right_on='Record ID')

# Drop missing sources
df_main_drop_missing = df_main.dropna(subset=['source'])

# Sort main subscription df by commit date
df_main_sorted = df_main_drop_missing.sort_values(by=['committed_since'])

# Set up dataframe for matching ids with inbound source
df_ids_sources = df_main_sorted.groupby('econ_id').agg({'econ_id':'first','source':'first'})

# Rename columns to allow for merging
df_ids_sources.rename(columns={'econ_id': 'id', 'source': 'inbound_source'}, inplace=True)

In [12]:
# Join sources with customer ids
df_main = df_main.merge(df_ids_sources[['id', 'inbound_source']], how='left', left_on='econ_id', right_on='id')

In [18]:
# Set up dataframe to be used for input to arr and carr calculations in other script
# Select relevant columns
df_main_columns = ['econ_id', 'total_item_price_EUR', 'committed_since', 'current_contract_start', 'current_contract_end', 'cancelled', 'renewal', 'region', 'fte_size_segment', 'inbound_source','product_group', 'product_type'] #'product_group', 'product_type', 
df_customer_data = df_main[df_main_columns]

# Rename columns
df_customer_data.rename(columns={'econ_id': 'id', 'total_item_price_EUR': 'value', 'committed_since': 'commit', 'current_contract_start': 'start', 'current_contract_end': 'end', 'fte_size_segment': 'size', 'product_group': 'product', 'product_type': 'type'}, inplace=True) #'product_group': 'product', 'product_type': 'type'

# Replace region names to correspond to arr/carr script
df_customer_data['region'].replace('Rest of World', 'ROW', inplace=True)

# Lower case all strings
df_customer_data = df_customer_data.applymap(lambda s:s.lower() if isinstance(s, str) else s)

# Output final dataframe
df_customer_data.to_excel('customer-subscription-data-dump-23-aug-01.xlsx', index=False)

/var/folders/84/np97p6s9023f4v7_n4n1nqk40000gp/T/ipykernel_39228/3414244019.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_customer_data.rename(columns={'econ_id': 'id', 'total_item_price_EUR': 'value', 'committed_since': 'commit', 'current_contract_start': 'start', 'current_contract_end': 'end', 'fte_size_segment': 'size', 'product_group': 'product', 'product_type': 'type'}, inplace=True) #'product_group': 'product', 'product_type': 'type'
/var/folders/84/np97p6s9023f4v7_n4n1nqk40000gp/T/ipykernel_39228/3414244019.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_customer_data['region'].replace('Rest of 

In [14]:
## ---=== END ===--- ##

In [15]:
df_customer_data = df_customer_data.applymap(lambda s:s.lower() if isinstance(s, str) else s)

In [16]:
df_main.to_excel('nov-main-to-check-2.xlsx')

In [17]:
df_main.to_excel('dec-renewals-all-subs.xlsx', index=False)

KeyboardInterrupt: 

In [ ]:
s_id_db = set(df_customer_data['id'])
s_id_firmo = set(df_firmographics['econ_id'])

In [ ]:
s_id_firmo.difference(s_id_db)

{6004,
 6005,
 6106,
 6108,
 6372,
 6524,
 6811,
 6813,
 6959,
 7075,
 1005916,
 272105381}

In [ ]:
s_id_db.difference(s_id_firmo)

{7093,
 7094,
 7095,
 7096,
 7097,
 7098,
 7099,
 7100,
 7101,
 7102,
 7103,
 7104,
 7105,
 7106,
 7107,
 7108,
 7109,
 7110,
 7111,
 7112}

In [ ]:
df_firmographics.columns

Index(['customer_name', 'econ_id', 'country', 'region', 'number_of_employees',
       'sub_industry', 'industry', 'industry_group', 'sector'],
      dtype='object')